In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from lifelines import ExponentialFitter, WeibullFitter, CoxPHFitter


In [22]:
%store -r df_final

In [23]:
df_final

,region,start_date,end_date,duration_quarters,event,GDP,Personal_Income
0,Centro,2008-07-01,2012-01-01,14.21,1,12.493620,12.192283
1,Este,2008-04-01,2012-04-01,16.23,1,13.017099,12.774096
2,Norte,2008-04-01,2012-01-01,15.22,1,12.759840,12.494906
3,Oeste,2008-04-01,2012-04-01,16.23,1,12.705061,12.404716
4,Sur,2008-07-01,2012-04-01,15.22,1,12.876164,12.595933


In [25]:
#Modelo parametrico

exp_model = ExponentialFitter()
exp_model.fit(df_final['duration_quarters'], event_observed=df_final['event'])
exp_model.print_summary()

<lifelines.ExponentialFitter:"Exponential_estimate", fitted with 5 total observations, 0 right-censored observations>
   number of observations = 5
number of events observed = 5
           log-likelihood = -18.68
               hypothesis = lambda_ != 0

---
         coef  se(coef)  coef lower 95%  coef upper 95%
lambda_ 15.42      6.90            1.90           28.94

         cmp to    z    p  -log2(p)
lambda_    0.00 2.24 0.03      5.30
---
AIC = 39.36

In [26]:
# Weibull
weibull_model = WeibullFitter()
weibull_model.fit(df_final['duration_quarters'], event_observed=df_final['event'])
weibull_model.print_summary()

<lifelines.WeibullFitter:"Weibull_estimate", fitted with 5 total observations, 0 right-censored observations>
   number of observations = 5
number of events observed = 5
           log-likelihood = -5.52
               hypothesis = lambda_ != 1, rho_ != 1

---
         coef  se(coef)  coef lower 95%  coef upper 95%
lambda_ 15.77      0.30           15.19           16.35
rho_    25.11      9.20            7.08           43.14

         cmp to     z      p  -log2(p)
lambda_    1.00 49.89 <0.005       inf
rho_       1.00  2.62   0.01      6.84
---
AIC = 15.04

In [33]:
cox = CoxPHFitter()
cox.fit(df_final, duration_col='duration_quarters', event_col='event', formula="GDP")
cox.print_summary()

<lifelines.CoxPHFitter: fitted with 5 total observations, 0 right-censored observations>
             duration col = 'duration_quarters'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 5
number of events observed = 5
   partial log-likelihood = -4.29
         time fit was run = 2025-10-30 15:36:54 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
GDP       -3.12      0.04      3.28           -9.54            3.30                0.00               27.22

           cmp to     z    p  -log2(p)
covariate                             
GDP          0.00 -0.95 0.34      1.55
---
Concordance = 0.75
Partial AIC = 10.59
log-likelihood ratio test = 0.99 on 1 df
-log2(p) of ll-ratio test = 1.64

In [31]:
df_final[['GDP','Personal_Income']].corr()

,GDP,Personal_Income
GDP,1.000000,0.998044
Personal_Income,0.998044,1.000000


In [37]:
base_cox = base_cox[base_cox['region']=='Centro']

base_cox[['GDP','Personal_Income']].corr()

,GDP,Personal_Income
GDP,1.000000,0.901986
Personal_Income,0.901986,1.000000
